<h1>Kapitel 6 Konfidensintervall</h1>

In [13]:
#importera paket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs 
    
# Importera Palmer Penguins
filepath = 'c:/users/davber/Python/notebooks/data/penguins.csv'
penguins = pd.read_csv(filepath)
penguins = penguins.dropna() # Plocka bort rader som innehåller NaN

In [14]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007


<h3>Exempel: Konfidensintervall för medelvärde</h3>
<strong> Konstruera ett 95% konfidensintervall för medelvikten hos pingviner av arten 'Adelie'.</strong> 

Eftersom $\sigma$ är okänd kommer vi behöva använda oss av t-statistika. Den beräknas enligt följande formel: 
<h3>${\bar x}\pm t_{\alpha/2}\frac{s}{\sqrt{n}}$</h3>
Vi börjar med att importera vårt t-fördelningspaket ur SciPy, och sedan subsetta vår datamängd så vi har det vi är intresserade av, Adelie-pingviner.

In [15]:
from scipy.stats import t # Importera t-fördelningen ur SciPy

adelie = penguins[penguins['species'] == 'Adelie'] # Subsetta data för species = Adelie

Sedan kan vi med hjälp av funktioner vi redan är bekanta med enkelt räkna ut värdet på vår t-statistika. Vi kan sedan konstruera själva konfidensintervallet genom att lägga till och dra från statistikans värde från vårt stickprovsmedelvärde.

In [21]:
mean = np.mean(adelie['body_mass_g']) # Beräkna medelvärde för stickprovet
std = np.std(adelie['body_mass_g'], ddof=1) # Beräkna standardavvikelse för stickprovet
n = len(adelie['body_mass_g']) # Beräkna n för stickprovet

alpha = 0.05 # Sätt signifikansgrad
t_crit = t.ppf(1-alpha/2, n-1) # Beräkna kritiskt t-värde
sem = std / np.sqrt(n) # Beräkna medelvärdesstandardfel 

upper = mean - t_crit * sem # Beräkna under gräns
lower = mean + t_crit * sem # Beräkna övre gräns

In [22]:
round(upper, 1), round(lower, 1) # Avrundning till 1 decimal

(3631.1, 3781.2)

In [24]:
print('Confidence interval (\u03B1=0.05) for average weight of \'Adelie\' penguins: ' + str(round(mean,1)) + ' \u00B1 ' + str(round(t_crit*sem,1))) # Printa konfidensintervall

Confidence interval (α=0.05) for average weight of 'Adelie' penguins: 3706.2 ± 75.0


<strong>Men, istället för att krångla till det med massa explicita beräkningar så kan vi använda oss av inbyggda funktioner i SciPy.</strong> SciPys t-modul har en schysst funktion för beräkning av konfidensintervall som heter interval(). Vi ger den helt enkelt vårt önskade konfidens (0.95, eller 1-alpha), antalet frihetsgrader, stickprovsmedelvärde, samt stickprovets standard error. Notera här att det är standard error ($s/\sqrt{n}$) som skall anges, och inte standardavvikelse ($s$). Standard error kan vi räkna ut m.h.a. funktionen <code>sem()</code> i SciPy.

In [29]:
sem = scs.sem(adelie['body_mass_g']) # Beräkna standard error med sem()

lower, upper = t.interval(confidence=1-alpha, df=n-1, loc=mean, scale=sem) # Beräkna undre samt övre gräns

print(f'Confidence interval (\u03B1=0.05) for average weight of \'Adelie\' penguins: {round(lower,1)}-{round(upper,1)}') # Printa konfidensintervall

Confidence interval (α=0.05) for average weight of 'Adelie' penguins: 3631.1-3781.2


<h3>Konfidensintervall för proportioner</h3>
<strong>Kosntruera ett 95% konfidensintervall för hur stor andel pingviner av typen 'Adelie' som lever på ön 'Torgersen'.</strong> 
Vi inser att uppgiften ber oss räkna ut en proportion, d.v.s. andelen av Adeliepingviner som återfinns på Torgersen-ön. Vi kommer då behöver beräkna test-statistikan för proportioner, som beräknas ur normalfördelningen (Lantz s. 172).
<h3>$\hat{p} \pm Z_{\alpha/2}\sqrt{\hat{p}(1-\hat{p})/(n-1)}$</h3>

Låt oss börja med att importera normalfördelningen, subsetta vår data, samt göra beräkningen "för hand" med kända funktioner ur NumPy och Pandas:

In [39]:
from scipy.stats import norm # Importera normalfördelningen ur SciPy

torgersen = adelie[adelie['island'] == 'Torgersen'] # Plocka ut Torgersen-pingviner ur 'adelie'

p_bar=len(torgersen)/len(adelie) # Beräkna p_bar, dvs gör en stickprovsskattning av proportionen p.
n=len(adelie) # beräkna storleken stickprovet
alpha=0.05 # Sätt konfidens

Test-statistikan $\hat{p}$ ovan gäller endast då $n\hat{p}\geq5$ och $n(1-\hat{p})\geq5$, så det är alltid bra att dubbelkolla så man inte är i en domän där ens approximation inte gäller.

In [40]:
n*p_bar, n*(1-p_bar) # Undersök värden på np och n(1-p) så att normalapproximationen gäller

(47.0, 99.0)

In [41]:
z_crit = norm.ppf(1-alpha/2) # Beräkna kritiskt Z-värde
sem = np.sqrt(p_bar*(1-p_bar)/(n-1)) # Beräkna medelvärdesstandardfel

upper = p_bar - z_crit*sem # Beräkna undre gräns
lower = p_bar + z_crit*sem # Beräkna övre gräns

round(upper, 2), round(lower, 2) # Avrunda till 2 decimaler

(0.25, 0.4)

In [42]:
print('Confidence interval (\u03B1=0.05) for proportion of Torgersen \'Adelie\' penguins relative to total \'Adelie\' penguins: ' + str(round(p_bar,2)) + ' \u00B1 ' + str(round(z_crit*sem,2)))

Confidence interval (α=0.05) for proportion of Torgersen 'Adelie' penguins relative to total 'Adelie' penguins: 0.32 ± 0.08


Precis som med konfidensintervallet för medelvärde kan vi göra samma sak lite enklare för oss genom att använda en inbyggd funktion. I det här fallet får vi hämta vår funktion ur statsmodels, då SciPy inte verkar ha någon inbyggd funktion för konfidensintervall för proportioner. <code>method='normal'</code> i det här fallet anger att vi har approximativt normalfördelade data.

In [38]:
from statsmodels.stats import proportion

lower, upper = proportion.proportion_confint(len(torgersen), len(adelie), alpha=alpha, method='normal')

print(f'Confidence interval (\u03B1=0.05) for proportion of Torgersen \'Adelie\' penguins relative to total \'Adelie\' penguins: {round(lower,2)}-{round(upper,2)}') # Printa konfidensintervall

Confidence interval (α=0.05) for proportion of Torgersen 'Adelie' penguins relative to total 'Adelie' penguins: 0.25-0.4
